# EXE Troubleshooting Notes

## Troubleshooting by Arcady

I am working on a Windows 11 computer and using the Powershell console.

## September 18, 2025

1. Generated an EXE of the project with Pyinstaller
2. Got **ModuleNotFoundError** on this line in main.py: `import qtmodern.styles`
3. Created a hooks directory within scripts
4. Created hooks\hook-qtmodern.py
5. Added qtmodern and PyQt5 to the Poetry venv (PyQt5 was a mistake as the project actually uses PyQt6; I fixed that later)
6. Updated main.spec to collect data files for qtmodern and PyQt5
7. Specified "qtmoderns" as an hidden import in main.spec
8. Removed and added qtmodern to the Poetry venv
9. Asked [a question](https://stackoverflow.com/questions/79767848/exe-created-with-pyinstaller-fails-to-import-qtmodern-as-specified-in-hook) on StackOverflow to get outside input
10. Got the virtual environment’s path and stored it in a variable called $venvpath in Powershell
11. Generated an EXE using main.py rather than main.spec (this makes Pyinstaller overwrite the main.spec file), with command `poetry run pyinstaller main.py --onefile --paths "$venvPath\Lib\site-packages"  --hidden-import=qtmodern  --hidden-import=qtmodern.styles --hidden-import=qtmodern.windows`
12. Got **ModuleNotFoundError** three lines further than before in main.py: `from event_connector import QtEvents, connect_decorated_methods, connect_event

```
#hook-qtmodern.py
from PyInstaller.utils.hooks import collect_data_files

datas = collect_data_files("qtmodern", includes=["**/*.qss"])

hiddenimports = [
    "qtmodern.styles",
    "qtmodern.windo
```wsin-hook